# TFM UOC SALINIDAD EN EL VALLE DEL GUADALHORCE

In [ ]:
#Se cargan las librerias que serán necesarias durante todo el desarrollo del script
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import datetime
from sklearn import tree
from datetime import datetime
#import pydotplus
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
#import pydotplus
from sklearn.preprocessing import MinMaxScaler
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
#import missingno as msno
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import SGD
from numpy import random

%matplotlib inline


# 1. Carga e integración de los datos

El primer paso que se deberán cargar e integrar los datos, ya que se encuentran dispersos en distintos ficheros.

A continuación se ejecuta un código que va recorriendo cada fichero CSV y va extrayendo la información de cada variable. Se crea una variable fecha, con el día, mes y año donde se ha tomada cada dato. 

Por lo tanto, crearemos un dataframe con todos los datos recogidos por la Junta de Andalucía en la explotación del sistema de presas del valle del guadalhorce. 

Adicionalmente se visualiza el dataset completo.

In [ ]:
#Se leen los ficheros csv correspondientes.

df_list=[]
for anyo in range(12,20):
    path_csv=f'dataset/ESTADILLO_DE_EMBALSES_20{anyo}.csv'
    df=pd.read_csv(path_csv,sep=";",encoding = 'unicode_escape',decimal=',',thousands='.')
    #Se consolidan las variables Dia, Mes y Anyo en una variable de nueva creación llamada Fecha
    df['aux']=df['Anyo'].astype(str) + df['Mes'].astype(str).str.zfill(2)+ df['Dia'].astype(str).str.zfill(2)
    df['Fecha'] = pd.to_datetime(df['aux'], format='%Y%m%d')
    #Eliminamos la columna auxilir 'aux'
    del df['aux']
    #Se cambia el formato de YYYY-MM-DD a DD-MM-YYYY
    df['Fecha'] = df['Fecha'].dt.strftime('%d-%m-%Y')
    #Movemos el campo fecha al comienzo del dataframe
    indice=df['Fecha']
    df.drop(labels=['Fecha'], axis=1,inplace = True)
    df.insert(0, 'Fecha', indice)
    if anyo==14:
        df.drop(df.columns[[74,75,76,77,78,79,80,81,82,83,84,85,86]],axis = 1, inplace = True)
    if anyo==15:
        df.drop(df.columns[[74,75,76,77,78,79,80,81,82,83,84,85,86]],axis = 1, inplace = True)
    df_list.append(df)

df_data=pd.concat(df_list,ignore_index = True)
#Se consolidan todos los dataframes de la lista en un solo dataframe. Así, quedará un dataframe que contenga los
#datos díarios dentro de un año.
    
#Se utiliza para visualizar todas las filas del dataframe creado
pd.set_option('display.max_rows', df_data.shape[0]+1)

In [ ]:
#Se visualiza el dataset completo
df_data.info()

# 2. Limpieza de los datos

En este apartado se detectan todos los missing values existentes en el dataset y se aplican distintas estrategias de imputación de la variable etiquetada.

Se realiza una imputación de NAN de la variable Salinidad_guadalhorce usando regresion logística.

También se limpia el dataset de las columnas vacias, carácteres inadecuados y se cambia el separador decimal de "," a ".".

In [ ]:
#Podemos visutalizar la cantidad de missing values presentes en el dataset (iterar para ver todas las columnas)
msno.matrix(df_data.iloc[:,25:50])

In [ ]:
#Porcentaje de NAN en la variable etiquetada
salinidad_mezcla_percentage = 100*df_data['Salinidad_mezcla'].isnull().sum()/len(df_data['Salinidad_mezcla'])
salinidad_mezcla_percentage

In [ ]:
#Eliminamos las columnas con todos los valores a 0
df_data=df_data.drop(df.columns[25], axis=1)
df_data=df_data.drop(df.columns[71], axis=1)
df_data=df_data.drop(df.columns[72], axis=1)
df_data=df_data.drop(df.columns[73], axis=1)

In [ ]:
#Se crea la variable df_dam que contiene todos los datos de la explotación de las presas del valle del guadalhorce
df_dam=df_data
#Se elimina un valor que está dando error
indexNames = df_dam[df_dam['Salinidad_mezcla'] == '1,170' ].index
df_dam.drop(indexNames , inplace=True)

#Ajustamos el tipo de la variable Salinidad_mezcla para poder procesarla
df_dam['Salinidad_mezcla']=df_dam['Salinidad_mezcla'].astype(str).astype(float)

In [ ]:
################-------------------------IMPUTACIÓN NAN EN SALINIDAD MEZCLA------------------###################

#Activar solamente un tipo de imputación.

#1) BORRAR LAS FILAS QUE CONTIENEN NAN
#df_dam.dropna(subset=['Salinidad_mezcla'],inplace=True)


#2) REGRESOR LINEAL PARA PREDECIR LOS VALORES DE NAN
#cols_sm=["Cota_guadalhorce","Cota_guadalteba","Volumen_total_guadalhorce_guadalteba_conde","Desembalse_total_guadalhorce_guadalteba_conde"
      #,"Salinidad_mezcla"]

#df_sm=df_dam[cols_sm]
#test_df_sm=df_sm[df_sm["Salinidad_mezcla"].isnull()]
#df_sm=df_sm.dropna()

#ynan_train_sm=df_sm["Salinidad_mezcla"]
#Xnan_train_sm=df_sm.drop("Salinidad_mezcla",axis=1)
#Xnan_test_sm=test_df_sm.drop("Salinidad_mezcla",axis=1)

#Recurrimos a la regresión logistica para predecir el valor de los NAN.
#lr_sm=LinearRegression()
#lr_sm.fit(Xnan_train_sm, ynan_train_sm)
#ynan_pred_sm=lr_sm.predict(Xnan_test_sm)

#Se sustituyen los valores NAN por sus predicciones
#df_dam.loc[df_dam.Salinidad_mezcla.isnull(),'Salinidad_mezcla']=ynan_pred_sm

#3) IMPUTACIÓN CON MEDIA
#df_dam['Salinidad_mezcla'].fillna((df_dam['Salinidad_mezcla'].mean()),inplace=True)

#4) IMPUTACIÓN CON MEDIANA (con este obtenemos mas que con la predicción)
#df_dam['Salinidad_mezcla'].fillna((df_dam['Salinidad_mezcla'].median()),inplace=True)

#5) IMPUTACIÓN CON ROLLING MEAN (y eliminación de los NANs restantes)
#window = 30 # 30 dias de observación
#df_dam['Salinidad_mezcla'].fillna((df_dam['Salinidad_mezcla'].rolling(window,min_periods=1,).mean()),inplace=True)
#df_dam.dropna(subset=['Salinidad_mezcla'],inplace=True)

#6) IMPUTACIÓN CON ROLLING MEDIAN
#window = 30 # 30 dias de observación
#df_dam['Salinidad_mezcla'].fillna((df_dam['Salinidad_mezcla'].rolling(window,min_periods=1,).median()),inplace=True)
#df_dam.dropna(subset=['Salinidad_mezcla'],inplace=True)

#7) IMPUTACIÓN CON INTERPOLACIÓN LINEAL
df_dam["Salinidad_mezcla"]=df_dam.Salinidad_mezcla.interpolate(method='linear')

#8) IMPUTACIÓN CON INTERPOLACIÓN CUADRATICA
#df_dam["Salinidad_mezcla"]=df_dam.Salinidad_mezcla.interpolate(method='quadratic')
#df_dam.dropna(subset=['Salinidad_mezcla'],inplace=True)

#9) IMPUTACIÓN CON INTERPOLACIÓN CUBICA
#df_dam["Salinidad_mezcla"]=df_dam.Salinidad_mezcla.interpolate(method='cubic')
#df_dam.dropna(subset=['Salinidad_mezcla'],inplace=True)

#10) IMPUTACIÓN CON INTERPOLACIÓN AKIMA
#df_dam["Salinidad_mezcla"]=df_dam.Salinidad_mezcla.interpolate(method='akima')
#df_dam.dropna(subset=['Salinidad_mezcla'],inplace=True)

#11) IMPUTACIÓN CON INTERPOLACIÓN SPLINE
#df_dam["Salinidad_mezcla"]=df_dam.Salinidad_mezcla.interpolate(method='spline',order=3)
#df_dam.dropna(subset=['Salinidad_mezcla'],inplace=True)

In [ ]:
#Se va usa unar regeresion logistica para calucular el valor de los NAN de la variable "Salinidad_guadalhorce"
#Planteamos dos escenarios en la variable "Salinidad_mezcla", quitando los NAN y sustituyendo por la media
#Implementamos por un lado KFold cross-validation y, posteriormente GridSearchCV

#Se va usa unar regeresion logistica para calucular el valor de los NAN de la variable "Salinidad_guadalhorce"

#Usamos un regresor lineal para predecir los valores NAN de la varibale Salinidad_guadalhorce.
cols=["Cota_guadalhorce","Salinidad_mezcla","Salinidad_guadalhorce"]

#Se pueden meter mas variables como "Variacion_volumen_guadalhorce", "Volumen_evaporado_guadalhorce" (esta ultima
#habria que quitarle los puntos y pasarla a float antes de meterla en el dataset de predicción)
df=df_dam[cols]
test_df=df[df["Salinidad_guadalhorce"].isnull()]
df=df.dropna()

ynan_train=df["Salinidad_guadalhorce"]
Xnan_train=df.drop("Salinidad_guadalhorce",axis=1)
Xnan_test=test_df.drop("Salinidad_guadalhorce",axis=1)

#Recurrimos a la regresión logistica para predecir el valor de los NAN.
lr=LinearRegression()
lr.fit(Xnan_train, ynan_train)
ynan_pred=lr.predict(Xnan_test)

#Se sustituyen los valores NAN por sus predicciones
df_dam.loc[df_dam.Salinidad_guadalhorce.isnull(),'Salinidad_guadalhorce']=ynan_pred

In [ ]:
#Porcentaje de NAN trás la imputación
salinidad_mezcla_na_percentage = 100*df_dam['Salinidad_mezcla'].isnull().sum()/len(df_dam['Salinidad_mezcla'])
salinidad_mezcla_na_percentage

# 3. Discretización de la variable etiquetada

La discretización de datos es el proceso de establecer varios puntos de corte para atributos con valores numéricos continuos con el fin de obtener valores enteros o discretos de dichos atributos.

En este caso, se ha contado con la Junta de Andalucía para realizar la asignación de intervalos:

-	Mezcla de aguas no salina: 0 ≤  Salinidad_mezcla ≤ 900: Salinidad_mezcla = 0 

-	Mezcla de aguas salina: Salinidad_mezcla > 900: Salinidad_mezcla = 1 


In [ ]:
#Discretiza la variable Salinidad_mezcla en 1 y 0. 0 para todos los valores por debajo de 900 y 1 para todos los
#valores por encima

#En este caso se ha subido a 900. 
bins=[0,900,5000]
df_dam['Salinidad_mezcla_tag'] = pd.cut(df_dam['Salinidad_mezcla'], bins, labels=[0,1])

#Sustituimos todos los valores NaN del df por 0
df_dam=df_dam.fillna(0)
#Borra la columna Salinidad_mezcla después de categorizarla
df_dam=df_dam.drop(['Salinidad_mezcla'],axis=1)

#Separamos la variable etiquetada del resto de dataset
df_dam_y=df_dam['Salinidad_mezcla_tag']
df_dam_x=df_dam.drop(df_dam[['Salinidad_mezcla_tag','Fecha','Hora','Horas_abastecimiento','Horas_ecologico',
                       'Horas_riego','Tiempo_desembalse_guadalhorce','Tiempo_desembalse_guadalteba',
                        'Volumen_central-reversible_tajo',
                        'Variacion_total_gaitanejo_tajo','Toneladas_sal_guadalhorce',
                        'Agua_sobrante','Toneladas_sal_mezcla']],axis=1)


#Eliminamos las comas que han quedado y algunos caracteres especiales
for i in range(len(df_dam_x.columns)):
    col = df_dam_x.columns[i]
    if df_dam_x[col].dtypes==object:
        df_dam_x[col] = df_dam_x[col].str.replace(',','.')
        df_dam_x[col] = df_dam_x[col].str.replace('#ÁREF!','0')
        df_dam_x[col] = df_dam_x[col].str.replace('V','0')
        df_dam_x[col] = df_dam_x[col].str.replace('VAR','0')
        df_dam_x[col] = df_dam_x[col].str.replace('var','0')
        df_dam_x[col] = df_dam_x[col].str.replace('0AR','0')
        df_dam_x[col] = df_dam_x[col].str.replace('12;0','0')
        df_dam_x[col] = df_dam_x[col].str.replace('9:00','0')
        df_dam_x[col] = df_dam_x[col].str.replace('-0.073','0')
        df_dam_x[col] = df_dam_x[col].str.replace('1.6-1.5','0')

#Convertimos algunas variables de "object" a "float"        
df_dam_x['Caudal_desembalse_guadalhorce']=df_dam_x['Caudal_desembalse_guadalhorce'].astype(float)
df_dam_x['Volumen_evaporado_guadalhorce']=df_dam_x['Volumen_evaporado_guadalhorce'].astype(float)
df_dam_x['Volumen_total_gaitanejo_tajo']=df_dam_x['Volumen_total_gaitanejo_tajo'].astype(float)
df_dam_x['Caudal_desembalse_guadalteba']=df_dam_x['Caudal_desembalse_guadalteba'].astype(float)
df_dam_x['Caudal_abastencimiento']=df_dam_x['Caudal_abastencimiento'].astype(float)
df_dam_x['Lluvia']=df_dam_x['Lluvia'].astype(float)        

#Convertimos los NAN a 0 de algunas variables, trás la conversión a float
df_dam_x['Caudal_desembalse_guadalhorce']=df_dam_x['Caudal_desembalse_guadalhorce'].fillna(0)
df_dam_x['Volumen_evaporado_guadalhorce']=df_dam_x['Volumen_evaporado_guadalhorce'].fillna(0)
df_dam_x['Volumen_total_gaitanejo_tajo']=df_dam_x['Volumen_total_gaitanejo_tajo'].fillna(0)
df_dam_x['Caudal_desembalse_guadalteba']=df_dam_x['Caudal_desembalse_guadalteba'].fillna(0)
df_dam_x['Caudal_abastencimiento']=df_dam_x['Caudal_abastencimiento'].fillna(0)
df_dam_x['Lluvia']=df_dam_x['Lluvia'].fillna(0)

#Vamos a normalizar todas las variables de df_dam_x usando MinMaxScaler.
scaler=MinMaxScaler()
norm_df=scaler.fit_transform(df_dam_x)

#Se vuelve crear un dataframe ya que al normalizar se realiza una conversión a numpy.ndarray
norm_df_dam_x=pd.DataFrame(norm_df,columns=list(df_dam_x.columns))

# 4. Reducción de dimensionalidad

Con el fin de ver cuales son las variables que mas influyen en la salinidad de la mezcla vamos a obtener los 20 atributos mas relevantes a través de SelectKBest

In [ ]:
#Features selection con SelectKBest y ANOVA

#Extraemos los nombres de las variables
features_list=list(norm_df_dam_x.columns)

#Aplicación de SelectKBest con el método ANOVA para seleccionar 20 variables mas influyentes
features_test = SelectKBest(score_func=f_classif, k=20)
fit = features_test.fit(norm_df_dam_x, df_dam_y)

set_printoptions(precision=4)
print(fit.scores_)
#Los valores nan corresponden a las variables "Filtraciones_guadalhorce","Filtraciones_guadalteba" y "Min_ecologico".
#Resultan nan porque son constantes en el caso de las dos primeras, y en el caso de Min_ecologico, todos los valores
#son 0, aunque al incio no se detecten como tal.
features = fit.transform(norm_df_dam_x)

In [ ]:
#Se imprimen las 20 variables mas relevantes
df_scores=pd.DataFrame(fit.scores_)
df_columns=pd.DataFrame(features_list)

feature_scores = pd.concat([df_columns, df_scores],axis=1)
feature_scores.columns = ['Feature_Name','Score']  
print(feature_scores.nlargest(20,'Score'))

# 5. Aplicación de métodos de ML 

Dado el conjunto de datos resultante de la fase de preprocesado, el siguiente paso que se pretende resolver, con la aplicación de los distintos métodos de aprendizaje automático, un problema de clasificación binaria de la variable Salinidad_mezcla. En este tipo de problemas se pretende encontrar un predictor o clasificador con el fin de predecir la clase de la variable etiquetada, en este caso, predecir la clase 0, correspondiente con muestras de agua no salina, o la clase 1, correspondiente con muestras de agua salina 

In [ ]:
#Se separan de forma aleatoria los conjuntos de entrenamiento y test según la regla 85%-15%.
X_train_gs, X_test_gs, y_train_gs, y_test_gs = train_test_split(df_dam_feat, df_dam_y, test_size=0.15,random_state=42)

In [ ]:
#Esta celda es usada para visualizar las fronteras de decisión.
#Se separan los conjuntos considerando únicamente las variables "Salinidad_guadalhorce" y "Cota_guadalhorce"
#Aqui no se usa cross-validation por lo tanto test set es de 30%
bound_df=df_dam_feat[['Cota_guadalhorce','Salinidad_guadalhorce']]
X_train_bound, X_test_bound, y_train_bound, y_test_bound = train_test_split(bound_df, df_dam_y, 
                                                                            test_size=0.10,random_state=42)

#Se crea la siguiente función para visualizar con los distintos algoritomos las fronteras de decisión
x_min, x_max = df_dam_feat['Cota_guadalhorce'].min()-0.1, df_dam_feat['Cota_guadalhorce'].max()+0.1
y_min, y_max = df_dam_feat['Salinidad_guadalhorce'].min()-0.1, df_dam_feat['Salinidad_guadalhorce'].max()+0.1  

def plot_decision_boundaries(x, y, labels, model, 
                             x_min=x_min, 
                             x_max=x_max, 
                             y_min=y_min, 
                             y_max=y_max, 
                             grid_step=0.02):
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, grid_step),
                         np.arange(y_min, y_max, grid_step))
    
    # Predecimos el classifier con los valores de la meshgrid.
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]

    # Hacemos reshape para tener el formato correcto.
    Z = Z.reshape(xx.shape)

    # Seleccionamos una paleta de color.
    arr = plt.cm.coolwarm(np.arange(plt.cm.coolwarm.N))
    arr_hsv = mpl.colors.rgb_to_hsv(arr[:,0:3])
    arr_hsv[:,2] = arr_hsv[:,2] * 1.5
    arr_hsv[:,1] = arr_hsv[:,1] * .5
    arr_hsv = np.clip(arr_hsv, 0, 1)
    arr[:,0:3] = mpl.colors.hsv_to_rgb(arr_hsv) 
    my_cmap = ListedColormap(arr)
    
    # Hacemos el gráfico de las fronteras de decisión.
    fig, ax = plt.subplots(figsize=(7,7))
    plt.pcolormesh(xx, yy, Z, cmap=my_cmap)

    # Añadimos los punts.
    ax.scatter(x, y, c=labels, cmap='coolwarm')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.grid(False)

## 5.5 Artificial Neural Network (ANN)

In [ ]:
#Recurrimos a la función definida en el ejercicio anterior, con la particularidad de que añadiremos una capa oculta
#a nuestra red neuronal y le pasaremos el número de neuronas que contendrá dicha capa oculta
def create_model_ann(learn_rate,units):
    # Construimoso el modelo a través de Keras. A partir de un modelo secuencial creado con Sequential, 
    #añadimos capas con Dense.
    model=Sequential()
    #Añadimos la capa de salida, indicandole que la dimensión de la entrada es 2 y que la función de activación es 
    #softmax. Se le indica que no se desea vector de bias.
    model.add(Dense(units, input_dim=20, activation='relu'))
    model.add(Dense(2, activation='softmax',use_bias=False,name="Output_layer"))
    #Se realiza un resumen del modelo obtenido.
    model.summary()
    # #Una vez que el modelo está definido, compilamos. A la hora de compilar le indicamos el optimizador a usar, que 
    #en este caso será SGD. Además se le indica que la métrica a tener en cuenta es la precisión, como en los ejemplos
    #anteriores. El atributo 'loss' se configura como 'sparse_categorical_crossentropy' para evitar el one-hot-enconding
    optimizer = SGD(lr=learn_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    return model

In [ ]:
#Se construye el vector de hiperparámetros elegidos aleatoriamente en el rango indicado
n_epoch=[50,75,100]
# n_epoch=[50]
lr=[0.1,0.5,0.9]
#lr=[0.1]
units=[10,30,40]
#units=[10]
param_grid_ann = dict(epochs=n_epoch,learn_rate=lr,units=units)

#Se crea el modelo a través de la función create_model2 definida mas arriba. Con el parámetro n_iter=20 le indicamos a la función que se
#usarán 20 combinaciones de hiperparámetros elegidos aleatoriamente.
model_ann=KerasClassifier(build_fn=create_model_ann)
#gscv_ann=GridSearchCV(model_ann,param_grid_ann, cv=10, n_jobs=-1,scoring='accuracy')
gscv_ann=GridSearchCV(model_ann,param_grid_ann, cv=10, n_jobs=1,scoring='accuracy',verbose=1)
#Se usa el conjunto de datos original (sin one-hot-encoded) ya que el atributo 'loss' se configuró como
#sparse_categorical_crossentropy, considerando a 'y' como atributo categorico.
gscv_result_ann=gscv_ann.fit(X_train_gs,y_train_gs)

#Una vez se evaluan el modelo de neural network para los hiperparámetros se obtienen la mejor precisión del modelo 
#y los hiperparámetros a partir de los cuales conseguimos dicho ratio.
print(gscv_result_ann.best_score_)
print(gscv_result_ann.best_params_)

In [ ]:
#Construimos el dataframe con el que se contruirá el heatmap
sc6b=gscv_result_ann.cv_results_['mean_test_score']
param6b=gscv_result_ann.cv_results_['params']
print(param6b)
gscv_result_ann.cv_results_
aux1_6b=pd.DataFrame(param6b)
aux2_6b=pd.DataFrame({'Accuracy':sc6b})
df_rnd6b=pd.concat([aux1_6b,aux2_6b],axis=1)
df_rnd6b

In [ ]:
#Probamos el poder predictor del clasificador hallado (best_estimator_)
y_pred_ann = gscv_result_ann.best_estimator_.predict(X_test_gs)

print("Accuracy:",metrics.accuracy_score(y_test_gs, y_pred_ann))

In [ ]:
#REFERENCIAS

#1. https://stackoverflow.com/questions/58072683/combine-year-month-and-day-in-python-to-create-a-date
#2. https://stackoverflow.com/questions/52291519/pandas-cant-convert-datetime-yyyy-mm-dd-to-dd-mm-yyyy
#3. https://stackoverflow.com/questions/25122099/move-column-by-name-to-front-of-table-in-pandas
#4. https://matthew-brett.github.io/teaching/string_formatting.html
#5. https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
#6. https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s
#7. https://www.delftstack.com/howto/python-pandas/how-to-delete-dataframe-row-in-pandas-based-on-column-value/
#8. https://stackoverflow.com/questions/37724225/how-to-categorize-floating-values-in-python-using-pandas-library
#9. https://www.youtube.com/watch?v=mnKm3YP56PY
#10.https://www.researchgate.net/publication/224196395_Feature_Selection_Using_Principal_Component_Analysis
#11.https://machinelearningmastery.com/feature-selection-machine-learning-python/
#12.https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/
#13.https://medium.com/@nmscott14/3-feature-selection-methods-e7ccd6dbf316
#14.https://towardsdatascience.com/predict-missing-values-in-the-dataset-897912a54b7b
#15.https://towardsdatascience.com/why-and-how-to-cross-validate-a-model-d6424b45261f
#16.https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7
#17.https://machinelearningmastery.com/statistical-imputation-for-missing-values-in-machine-learning/